<a href="https://colab.research.google.com/github/laprass/drive/blob/main/%5Bn122a%5D_chi_square_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [55]:
!pip install --upgrade scipy

     |████████████████████████████████| 27.4MB 150kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [40]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats

In [92]:
# 1/4 : 데이터 불러오기
url = 'https://raw.githubusercontent.com/laprass/drive/main/house.csv'
df = pd.read_csv(url, encoding='utf-8')
df = df.reindex(index=[0,3,2,1]) # 서울 대전 대구 부산 순서로 행순서 변경 (https://k-glory.tistory.com/21)
df = df.set_index('시도') # 지역으로 인덱스 지정
df

,60,60-85,85
시도,,,
서울,54,2,0
대전,782,1,0
대구,143,1437,44
부산,577,735,142


In [93]:
print(df.shape)
df
# 1/4 완료 (2/7)

(4, 3)


,60,60-85,85
시도,,,
서울,54,2,0
대전,782,1,0
대구,143,1437,44
부산,577,735,142


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

9월달 데이터를 기준으로 한다면
```python
 [0+0+0+52+2+0, 0+0+0+590+665+142, 0+0+0+113+1061+42, 0+0+0+772+1+0]
```
을 비교해야할겁니다.




In [43]:
# 2/4 : 원사이드 카이제곱 검정, 결과값은 'chi1'
## 카이제곱 공부하기 : https://blog.naver.com/moses3650/221192466252

## Ho : 지역에 따라 미분양수에 차이는 없다.
## H1 : 차이가 있다!

# 행단위 데이터 읽어오기 : https://devpouch.tistory.com/47
# df.iloc[0] 

In [72]:
## 지역별 기대값(평균) 구해서 데이터프레임으로 만들기
#참고 코드 : https://nittaku.tistory.com/113
# skipna = True : 결측값 없을때, 있으면 false

df1 = df[['60','60-85','85']]
df1['total'] = df['60']+df['60-85']+df['85']
df1

,60,60-85,85,total
시도,,,,
서울,54,2,0,56
대전,782,1,0,783
대구,143,1437,44,1624
부산,577,735,142,1454


In [73]:
stats.chisquare(df1['total']) # 원래는 ()안에 관측값과 기대값이 (A,B) 형태로 들어가야된다. 
                              # 과제에서는 기대값이 관측값과 동일(합산)하므로 1개만 입력함

Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

In [46]:
## p값이 0.0이므로 귀무가설은 기각, 대립가설 채택
## 검증결론은 '지역별로 차이가 있다'

In [74]:
chi1 = stats.chisquare(df1['total'])
round(chi1.statistic, 3)
# 2/4 완료 

1564.457

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

9월달 데이터를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [48]:
# 3/4 이원카이제곱검정, 결과값은 chi2

## Ho : 지역과 규모 간에 관계가 없을 것이다.
## H1 : 관계 있다! (=두 변수가 독립적이지 않다.)

In [75]:
df2= df1.drop(['total'], axis=1)

In [76]:
stats.chi2_contingency(df2) 

# 출력결과는 순서대로 (통계량, p값, 자유도 의미)

(2064.5767314171994, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
        [311.04110288, 434.77789124,  37.18100587],
        [645.12228746, 901.76155221,  77.11616033],
        [577.59101353, 807.36533061,  69.04365586]]))

In [77]:
chi2 = stats.chi2_contingency(df2) 
chi2[0]

2064.5767314171994

In [52]:
## p값이 0.0이므로 귀무가설 기각, 대립가설 채택
## 분양지역과 분양규모는 서로 독립적이지 않으며, 유기적관계로 추정할 수 있다.
# 3/4 완료

### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

이 후 `chi2_contingency()`의 결과물과 비교해보세요

In [78]:
# 4/4 numpy만 사용해서 카이제곱 시행, 지역에 대한 일원 카이검정과 비교
# obs = 관측치 / exp = 기대값 / chi = 카이제곱값

In [79]:
# 관측치를 저장한다.

obs = df1['total']
obs

시도
서울      56
대전     783
대구    1624
부산    1454
Name: total, dtype: int64

In [80]:
# 기대값(평균)을 추가한다.

total_sum = df1['total'].sum() # 전체 미분양 총합
total_count = len(df1['total']) # 지역 갯수 카운트 https://rfriend.tistory.com/450
exp = total_sum / len(df1['total']) # 기대값의 총합에서 데이터 갯수를 나눠 평균 구하기
exp

979.25

In [107]:
# # numpy를 사용해 카이제곱 값을 구한다. https://nittaku.tistory.com/108

sl = np.square(obs[0] - exp) / exp # 서울의 카이제곱
dj = np.square(obs[1] - exp) / exp # 대전의 카이제곱
dg = np.square(obs[2] - exp) / exp # 대구의 카이제곱
bs = np.square(obs[3] - exp) / exp # 부산의 카이제곱

chi = (sl+dj+dg+bs) # 카이제곱의 합
chi

1564.4572376818994

## Grading
이전 과제를 완료했다면, 아래의 코드를 실행하세요.

In [82]:
def grade(df, chi1, chi2, obs, exp, chi2_contingency):
  score = 0
  score += (df.shape==(4,3)) # 1
  score += (df.sum().sum() == 3917) # 1-2
  score += (round(chi1.statistic, 3) == 1564.457) # 2
  score += (round(chi2[0], 3) == 2064.577) # 3
  score += ((obs == [56, 783, 1624, 1454]).sum() == 4) # 4-1
  score += (exp == 979.25) # 4-2
  score += (round(chi, 3) == 1564.457) # 4-3
  return(score)

In [94]:
grade(df, chi1, chi2, obs, exp, chi)

7

## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [66]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]
def myChisq(value):

# 내용을 채우세요.

myChisq(v1) 

# chisquare값과 pvalue를 출력해야합니다.

myChisq(v2)

IndentationError: ignored

### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



## 3. Sign Test

아래 링크를 참조하여 Sign Test에 대한 글을 읽고

<https://en.wikipedia.org/wiki/Sign_test>

다음 데이터에서 두 그룹간에 차이가 있다에 대한 Sign Test의 과정과 결과를 설명하세요

|A|B|
|:-:|:-:|
|142|138|
|140|136|
|144|147|	
|144|139|	
|142|143|	
|146|141|	
|149|143|	
|150|145|	
|142|136|	
|148|146|	

